In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os,cv2
import json
from IPython.display import Image
from keras.preprocessing import image
from keras import optimizers
from keras import layers,models
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
from keras import regularizers
from keras.models import Sequential, Model 
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from tqdm import tqdm, tqdm_notebook
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint



In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
PATH = "../input"
!ls {PATH}

In [ ]:
train_dir = "../input/train/train"
test_dir = "../input/test/test"

In [ ]:
df_train = pd.read_csv(f'{PATH}/train.csv',low_memory=False)
df_test = pd.read_csv(f'{PATH}/sample_submission.csv',low_memory=False)
df_train.has_cactus= df_train.has_cactus.astype(str)

In [ ]:
print('Shape of Training data: {}'.format(df_train.shape))
print('Features: {}'.format(df_train.columns))

In [ ]:
df_train.tail()

In [ ]:
plt.figure(figsize = (6,5))
sns.set(style="darkgrid")
ax = sns.countplot(x = 'has_cactus',hue='has_cactus',data = df_train)
plt.xticks(rotation='vertical')
plt.xlabel('Count of Each Category', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

In [ ]:
df_train['has_cactus'].value_counts()

In [ ]:
im = cv2.imread("../input/train/train/01e30c0ba6e91343a12d2126fcafc0dd.jpg")
plt.imshow(im)

Data Augmentation: https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1/.255)

In [ ]:
batch_size= 32

In [ ]:
train_generator = train_datagen.flow_from_dataframe(df_train[:15000], directory=train_dir, x_col='id', y_col='has_cactus', 
                    target_size=(150, 150), color_mode='rgb', classes=None, class_mode='binary',
                    batch_size=batch_size, 
                    shuffle=True, seed=None, 
                    save_to_dir=None, save_prefix='', save_format='png', 
                    subset=None, interpolation='nearest', drop_duplicates=True)

In [ ]:
valid_generator = valid_datagen.flow_from_dataframe(df_train[15000:], directory=train_dir, x_col='id', y_col='has_cactus', 
                    target_size=(150, 150), color_mode='rgb', classes=None, 
                    class_mode='binary', batch_size=batch_size, 
                    shuffle=True, seed=None, 
                    save_to_dir=None, save_prefix='', save_format='png', 
                    subset=None, interpolation='nearest', drop_duplicates=True)

In [ ]:
vgg16_net = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=(150, 150, 3))

In [ ]:
vgg16_net.trainable = False
vgg16_net.summary()

In [ ]:
model1 = Sequential()
model1.add(vgg16_net)
model1.add(Flatten())
model1.add(Dense(256))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))

In [ ]:
model1.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3), 
              metrics=['accuracy'])

In [ ]:
batch_size = 32
nb_epoch = 1000

Transfer Learning with Vgg_16 and data augmentation is getting bad accuracy. Remove Augmentation.
Augmentation edited with keras defaults - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and fit_generator edited.

In [ ]:
%%time
history = model1.fit_generator(train_generator,validation_data = valid_generator,validation_steps=800,
                              epochs=50,
                              steps_per_epoch=2000 // batch_size,
                              verbose=2)

In [ ]:
model1.save_weights('first_try_vgg16_net.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Data Augmentation - Without using Transfer Learning

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
#optim = Adam(lr=0.0022, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
filepath = "best_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
call_backs_list = [checkpoint]

In [ ]:
%%time

history = model.fit_generator(train_generator,validation_data = valid_generator,validation_steps=800,
                              epochs=50,
                              steps_per_epoch=2000 // batch_size,
                              verbose=2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()